# Create a feedback ZIP for Moodle based on Gradescope exported submissions.

In [ ]:
from pathlib import Path
import pandas as pd
import shutil
import yaml

In [ ]:
base_dir = Path('/Users/ka37/Dropbox/Mac/Downloads/assignment_2778979_export 2')
metadata_file = base_dir / 'submission_metadata.yml'

In [ ]:
metadata = yaml.load(metadata_file.read_text(), Loader=yaml.FullLoader)
next(iter(metadata.values()))

In [ ]:
# load the Moodle offline grading spreadsheet for this assignment
# this is needed because Moodle has participant ids that don't match the "ID number" in the roster
roster = pd.read_csv('/Users/ka37/Dropbox/Mac/Downloads/Grades-23SP CS-344-A-Midterm 1-1618120.csv')
roster.head()

In [ ]:
# split out the user id from the "Identifier" column, which has strings like "Participant 1234"
roster['user_id'] = roster['Identifier'].str.split(' ').str[1].astype(int)
roster.head()

In [ ]:
out_dir = Path('../student_data/midterm_1')
out_dir.mkdir(exist_ok=True)

In [ ]:
for filename, submission_data in metadata.items():
    assert len(submission_data[':submitters']) == 1
    submitter = submission_data[':submitters'][0]
    name = submitter[':name']
    email = submitter[':email'].replace('@students.calvin.edu', '@calvin.edu')
    # find the student in the roster.
    student = roster[roster['Email address'] == email]
    assert len(student) == 1
    student = student.iloc[0]
    # create the output directory.
    student_dir = out_dir / f"{name}_{student['user_id']}_assignsubmission_file_"
    student_dir.mkdir(exist_ok=True)
    # copy the submission file.
    submission_file = base_dir / filename
    shutil.copy(submission_file, student_dir / 'feedback.pdf')